# Dessa vez eu mudei um pouco a lógica da animação que vinha usando

In [9]:
import os
import matplotlib.pyplot as plt
from matplotlib.animation import FuncAnimation
import random

# Opcional: para salvar vídeo
try:
    import imageio.v2 as imageio
except ImportError:
    imageio = None


def orientation(a, b, c):
    """Calcula o produto vetorial de AB e AC. 
    Útil para determinar se o ponto C está à esquerda, direita ou colinear a AB."""
    return (b[0] - a[0]) * (c[1] - a[1]) - (b[1] - a[1]) * (c[0] - a[0])


def build_initial_hulls(points):
    """Cria cascos iniciais a partir dos pontos: pares, ou trios se ímpar."""
    points = sorted(points)
    hulls = []
    n = len(points)
    
    i = 0
    while i < n:
        if n - i == 3:
            group = points[i:i+3]
            i += 3
        else:
            group = points[i:i+2]
            i += 2
        hulls.append(build_monotone_chain(group))
    return hulls


def build_monotone_chain(points):
    """Algoritmo de Monotone Chain para pequenos conjuntos de pontos."""
    if len(points) <= 1:
        return points

    pts = sorted(points)
    lower = []
    for p in pts:
        while len(lower) >= 2 and orientation(lower[-2], lower[-1], p) <= 0:
            lower.pop()
        lower.append(p)
    
    upper = []
    for p in reversed(pts):
        while len(upper) >= 2 and orientation(upper[-2], upper[-1], p) <= 0:
            upper.pop()
        upper.append(p)
    
    # Junta, evitando duplicatas nos extremos
    return lower[:-1] + upper[:-1]


def merge_hulls(left, right):
    """Une dois cascos convexos encontrando tangentes superiores e inferiores."""
    def is_lower(i, j):
        a, b = left[i], right[j]
        return all(orientation(a, b, p) >= 0 for p in left) and all(orientation(b, a, p) <= 0 for p in right)

    def is_upper(i, j):
        a, b = left[i], right[j]
        return all(orientation(a, b, p) <= 0 for p in left) and all(orientation(b, a, p) >= 0 for p in right)

    lower = upper = None
    for i in range(len(left)):
        for j in range(len(right)):
            if lower is None and is_lower(i, j):
                lower = (i, j)
            if upper is None and is_upper(i, j):
                upper = (i, j)
            if lower and upper:
                break
        if lower and upper:
            break
    
    i_low, j_low = lower
    i_up, j_up = upper

    # Caminha pelos cascos para formar o novo casco completo
    merged = []
    idx = i_low
    merged.append(left[idx])
    while idx != i_up:
        idx = (idx + 1) % len(left)
        merged.append(left[idx])

    idx = j_up
    merged.append(right[idx])
    while idx != j_low:
        idx = (idx + 1) % len(right)
        merged.append(right[idx])

    return merged


def divide_and_conquer(points, plot_steps=False, save_folder='Simulacoes', video_name='animation.mp4', fps=1):
    """Executa algoritmo divide-and-conquer para construir o casco convexo."""
    history = []
    hulls = build_initial_hulls(points)
    history.append([h.copy() for h in hulls])

    while len(hulls) > 1:
        new_hulls = []
        for i in range(0, len(hulls), 2):
            if i + 1 < len(hulls):
                merged = merge_hulls(hulls[i], hulls[i+1])
                new_hulls.append(merged)
            else:
                new_hulls.append(hulls[i])
        hulls = new_hulls
        history.append([h.copy() for h in hulls])

    if plot_steps:
        os.makedirs(save_folder, exist_ok=True)
        for step, hs in enumerate(history):
            plt.figure()
            for hull in hs:
                xs = [p[0] for p in hull] + [hull[0][0]]
                ys = [p[1] for p in hull] + [hull[0][1]]
                plt.plot(xs, ys, marker='o')
            plt.title(f"Iteração {step}")
            plt.axis('equal')
            plt.savefig(os.path.join(save_folder, f"step_{step}.png"))
            plt.close()

        if imageio:
            frames = [imageio.imread(os.path.join(save_folder, f"step_{i}.png")) for i in range(len(history))]
            imageio.mimsave(os.path.join(save_folder, video_name), frames, fps=fps)

    return hulls[0]  # retorna o casco convexo final

# Para testar o algoritmo, mude a váriavel "pts" abaixo e rode o algoritmo. 
Se quiser uma simulação visual com o matplotlib, mude a variável "animated" para "True". Se quiser gerar pontos aleatórios, utilize a função "generate_random_points" como pode ser visto abaixo. Caso contrário, passe uma array de pontos no formato [(a,b), (c,d), (e,f), ... (y,z)]

In [10]:
random.seed(42)
def generate_random_points(num_points):
    return [(random.randint(-50, 50), random.randint(-50, 50)) for _ in range(num_points)]

In [11]:
# ESSAS SÃO AS VARIÁVEIS QUE VOCÊ DEVE MUDAR
pts = generate_random_points(50)
animated = True

if __name__ == '__main__':
    history = divide_and_conquer(pts, plot_steps=animated)
    print("Pontos do casco final:", history)

Pontos do casco final: [(-15, -50), (-46, -47), (-47, 44), (-21, 48), (36, 44), (41, 33), (47, -7), (47, -30), (44, -37), (40, -42), (27, -47)]
